DOWNLOAD ALL NECESSARY FILE FROM THE FOLDER

In [ ]:
import pywt
from numpy import convolve
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
%matplotlib ipympl

In [ ]:
texts0 = [[], []]
texts1 = [[], []]
texts2 = [[], []]

with open("/home/muhammadakbarmaulana/Documents/Untitled Document 1.txt", "r") as file:
  textfile = file.readlines()

for text in textfile:
    text = text.split('\t')
    text[0] = text[0].split(':')
    text[0] = text[0][2]
    text[2] = text[1].replace('\n',"")
    texts0[0].append(float(text[0]))
    texts0[1].append(float(text[1]))

with open("data.txt", "r") as file:
  textfile = file.readlines()

for text in textfile:
    text = text.split('\t')
    text[1] = text[1].replace('\n',"")
    texts1[0].append(float(text[0]))
    texts1[1].append(float(text[1]))

with open("Print_13_v2_PCG_RV.txt", "r") as file:
  textfile = file.readlines()

for text in textfile:
    text = text.split('\t')
    text[1] = text[1].replace('\n',"")
    texts2[0].append(float(text[0]))
    texts2[1].append(float(text[1]))

In [ ]:
def continuouswavelettransform(var, waveletfunction, waveletscale):
    pass

def discretewavelettransfrom(var, waveletfunction, scalingfunction):
    """Calculate the discrete wavelet transform using convolution

    Parameters
    ----------
    var : 1D Array
        Input signal for transform
    waveletfunction : 1D Array
        Wavelet function for the transform, can be picked from the waveletsignalfunction
    scalingfunction : 1D Array
        Scaling function for the transform, can be picked from the scalingsignalfunction

    Returns
    -------
    [1D Array, 1D Array]
        The first array is the detailed coefficient(highpassfilter), and the second is the approximation coefficient(lowpassfilter)
    """
    return convolve(var, waveletfunction, "same"), convolve(var, scalingfunction, "same")

def waveletsignalfunction(wavelettype, waveletlength, waveletscale):
    """Calculate the wavelet signal function for the wavelet transform

    Parameters
    ----------
    wavelettype : String
        Based on the selection on the screen widget, the wavelet type consist of the discrete and continuous wavelet 
    waveletlength : Integer
        Length of the wavelet in sequence
    waveletscale : Integer
        Scale of of the discrete wavelet, the elgnth of the wavelet determined by the 2**waveletscale

    Returns
    -------
    [1D Array, 1D Array]
        The first array return the x axis of the wavelet, the second is the wavelet, both have the same length.
    """
    if wavelettype == "DWT-Haar":
        wavelet = pywt.Wavelet("haar")
        _, waveletfunction, x = wavelet.wavefun(level=waveletscale)
        return x, waveletfunction
    elif wavelettype == "DWT-Symlet-4":
        wavelet = pywt.Wavelet("sym4")
        _, waveletfunction, x = wavelet.wavefun(level=waveletscale)
        return x, waveletfunction
    elif wavelettype == "CWT-Morlet":
        wavelet = pywt.ContinuousWavelet("morl")
        waveletfunction, x = wavelet.wavefun(level=waveletscale)
        return x, waveletfunction
    elif wavelettype == "CWT-Mexican Hat":
        wavelet = pywt.ContinuousWavelet("mexh")
        waveletfunction, x = wavelet.wavefun(level=waveletscale)
        return x, waveletfunction

def scalingsignalfunction(wavelettype, waveletlength, waveletscale):
    """Calculate the scaling function that act like lowpassfilter for the discrete wavelet transform

    Parameters
    ----------
    wavelettype : String
        Based on the selection on the screen widget, the wavelet type consist of the discrete and continuous wavelet 
    waveletlength : Integer
        Length of the scaling function in sequence
    waveletscale : Integer
        Scale of of the scaling function, the length of the wavelet determined by the 2**waveletscale

    Returns
    -------
    [1D Array, 1D Array]
        The first array return the x axis of the scaling function, the second is the scaling, both have the same length.
    """
    if wavelettype == "DWT-Haar":
        wavelet = pywt.Wavelet("haar")
        scalingfunction, _, x = wavelet.wavefun(level=waveletscale)
        return x, scalingfunction
    elif wavelettype == "DWT-Symlet-4":
        wavelet = pywt.Wavelet("sym4")
        scalingfunction, _, x = wavelet.wavefun(level=waveletscale)
        return x, scalingfunction


In [ ]:
def on_change(change):
    if selectwavelettype.value == None:
        return

    global temporaryconvolvedsignal
    global texts0
    temporaryconvolvedsignal = texts0
    waveletfunction = []
    scalingfunction = []
    
    with out:
        fig1.clear()
        ax1 = fig1.add_axes([0.12,0.1,0.85,0.85], xlabel="sequence", ylabel="amplitude", title="Signal")

        fig2.clear()
        ax2 = fig2.add_axes([0.12,0.1,0.85,0.85], xlabel="sequence", ylabel="amplitude", title="Transform Signal")

        try:
            xaxis, waveletfunction= waveletsignalfunction(
                selectwavelettype.get_interact_value(), 
                selectwaveletlength.get_interact_value(), 
                selectwaveletscale.get_interact_value())
            ax1.plot(xaxis, waveletfunction, label="Wavelet Function")
            ax1.legend()
            selectwaveletlength.value = len(xaxis)
        except Exception as e:
            pass
        
        try:
            xaxis, scalingfunction= scalingsignalfunction(
                selectwavelettype.get_interact_value(), 
                selectwaveletlength.get_interact_value(), 
                selectwaveletscale.get_interact_value())
            ax1.plot(xaxis, scalingfunction, label="Scaling Function")
            ax1.legend()
            selectwaveletlength.value = len(xaxis)
        except Exception as e:
            pass

        if selectwaveletlength.value >= len(temporaryconvolvedsignal[1]):
            out.clear_output()
            print('Length of the wavelet cannot exceed the input length signal! ({})'.format(len(temporaryconvolvedsignal[1])))
            return

        convolvedwaveletsignal, _ = discretewavelettransfrom(temporaryconvolvedsignal[1], waveletfunction, scalingfunction)
        ax2.plot(temporaryconvolvedsignal[0], convolvedwaveletsignal, label="Convolved Signal")
        out.clear_output()

def on_change_continuouswavelet(change):
    if selectwavelettype.value == None:
        return

    waveletfunction = []
    with out:
        fig1.clear()
        ax1 = fig1.add_axes([0.12,0.1,0.85,0.85], xlabel="sequence", ylabel="amplitude", title="Signal")

        fig2.clear()
        ax2 = fig2.add_axes([0.12,0.1,0.85,0.85], xlabel="sequence", ylabel="amplitude", title="Transform Signal")

        try:
            xaxis, waveletfunction= waveletsignalfunction(
                selectwavelettype.get_interact_value(), 
                selectwaveletlength.get_interact_value(), 
                selectwaveletscale.get_interact_value())
            ax1.plot(xaxis, waveletfunction, label="Wavelet Function")
            ax1.legend()
            selectwaveletlength.value = len(xaxis)
        except Exception as e:
            pass

        convolvedwaveletsignal, _ = continuouswavelettransform(temporaryconvolvedsignal[1], waveletfunction, selectwaveletscale.value())
        ax2.plot(temporaryconvolvedsignal[0], convolvedwaveletsignal, label="Convolved Signal")
        out.clear_output()

def on_button_clicked(_):
    if selectwavelettype.value == None:
        return

    global temporaryconvolvedsignal
    waveletfunction = []
    scalingfunction = []

    with out:
        fig1.clear()
        ax1 = fig1.add_axes([0.12,0.1,0.85,0.85], xlabel="sequence", ylabel="amplitude", title="Signal")

        fig2.clear()
        ax2 = fig2.add_axes([0.12,0.1,0.85,0.85], xlabel="sequence", ylabel="amplitude", title="Transform Signal")
        selectwaveletscale.value = selectwaveletscale.get_interact_value()+1

        try:
            xaxis, waveletfunction= waveletsignalfunction(
                selectwavelettype.get_interact_value(), 
                selectwaveletlength.get_interact_value(), 
                selectwaveletscale.get_interact_value())
            ax1.plot(xaxis, waveletfunction, label="Wavelet Function")
            ax1.legend()
            selectwaveletlength.value = len(xaxis)
        except Exception as e:
            pass
        
        try:
            xaxis, scalingfunction= scalingsignalfunction(
                selectwavelettype.get_interact_value(), 
                selectwaveletlength.get_interact_value(), 
                selectwaveletscale.get_interact_value())
            ax1.plot(xaxis, scalingfunction, label="Scaling Function")
            ax1.legend()
            selectwaveletlength.value = len(xaxis)
        except Exception as e:
            pass

        if selectwaveletlength.value >= len(temporaryconvolvedsignal[1]):
            out.clear_output()
            print('Length of the wavelet cannot exceed the input length signal! ({})'.format(len(temporaryconvolvedsignal[1])))
            return
        convolvedwaveletsignal, temporaryconvolvedsignal[1] = discretewavelettransfrom(temporaryconvolvedsignal[1], waveletfunction, scalingfunction)
        ax2.plot(temporaryconvolvedsignal[0], convolvedwaveletsignal, label="Convolved Signal")
        out.clear_output()
        print('Using this button enabling convolution of the scaling function to the signal thus making it increasing in level!')

def on_button_clicked_clearselection(_):
    with out:
        try:
            fig1.clear()
            fig2.clear()
            selectwavelettype.value = None
            selectwaveletlength.value = 0 
            selectcontinuouswavelettype.value = None
        except Exception as e:
            pass


        

In [ ]:
selectwavelettype = widgets.Select(
    options=['DWT-Haar', "DWT-Symlet-4"],
    value=None,
    description='Wavelet Type:',
    disabled=False
)

selectcontinuouswavelettype = widgets.Select(
    options=["CWT-Morlet", "CWT-Mexican Hat"],
    value=None,
    description="Continuous Wavelet Type:",
    disabled=False
)

selectwaveletlength = widgets.IntText(
    description='Wavelet Length:',
    disabled=False
)

selectwaveletscale = widgets.FloatSlider(
    value=5,
    min=0,
    max=15.0,
    step=0.5,
    description='Wavelet Refinement Scale:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

buttonnextwaveletscale = widgets.Button(
    description='Next Wavelet Scale')

buttonclearselection = widgets.Button(
    description='Clear')

out = widgets.Output()

selectwavelettype.observe(on_change)
selectcontinuouswavelettype.observe(on_change_continuouswavelet)
selectwaveletscale.observe(on_change)
buttonnextwaveletscale.on_click(on_button_clicked)
buttonclearselection.on_click(on_button_clicked_clearselection)

In [ ]:
fig0 = plt.figure()
ax0 = fig0.add_axes([0.1,0.1,0.85,0.85], xlabel="sequence", ylabel="amplitude", title="ElectroCardioGram Signal")
ax0.plot(texts0[0], texts0[1], label="ECG Signal")
# ax0.plot(texts1[0], texts1[1], label="PCG Signal", alpha=0.5)
ax0.legend()
plt.show()

temporaryconvolvedsignal = texts0 # store temporary data for leveling on DWT

In [ ]:
fig1 = plt.figure()
ax1= fig1.add_axes([0.12,0.1,0.85,0.85], xlabel="sequence", ylabel="amplitude", title="Wavelet Signal")

display(selectwavelettype, selectcontinuouswavelettype, selectwaveletlength, selectwaveletscale)

fig2 = plt.figure()
ax2 = fig2.add_axes([0.12,0.1,0.85,0.85], xlabel="sequence", ylabel="amplitude", title="Transform Signal")
plt.show()

display(buttonnextwaveletscale, buttonclearselection, out)
